In [15]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

# Function to generate and save augmented images
def generate_and_save_images(input_dir, output_dir):
    # Get the list of image files in the input directory
    image_files = os.listdir(input_dir)

    # Create the output directory
    os.makedirs(output_dir, exist_ok=True)

    # Generate augmented images
    for i, image_file in enumerate(image_files):
        image_path = os.path.join(input_dir, image_file)

        # Load the original image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Perform augmentation operations on the original image
        # Example: You can apply transformations like rotation, scaling, or adding noise here
        augmented_image = image  # Placeholder for augmentation operations

        # Save the augmented image
        augmented_image_path = os.path.join(output_dir, f"augmented_image_{i}.png")
        cv2.imwrite(augmented_image_path, augmented_image)

# Example usage
input_dir = "./data/images/boron"  # Path to the directory containing the original dataset
output_dir = "./data/images/augmented_images"  # Output directory for augmented images

generate_and_save_images(input_dir, output_dir)


In [2]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Reshape, Conv2DTranspose, Conv2D, Flatten
from keras.optimizers import Adam

def define_generator_model(input_shape):
    model = Sequential()
    model.add(Dense(256, input_dim=100))
    model.add(Dense(np.prod(input_shape)))
    model.add(Reshape(input_shape))
    model.add(Conv2DTranspose(128, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv2DTranspose(64, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv2DTranspose(32, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv2DTranspose(3, kernel_size=3, padding='same', activation='sigmoid'))
    return model

def define_discriminator_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv2D(128, kernel_size=3, padding='same', activation='relu'))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

input_directory = './data/images/boron'
output_directory = './data/images/cGAN_augmented'

# Get the dimensions of the original images
image_file = os.listdir(input_directory)[0]
image_path = os.path.join(input_directory, image_file)
image = cv2.imread(image_path)
image_shape = image.shape

# Load the generator model
generator_model = define_generator_model(image_shape)
generator_model.compile(loss='binary_crossentropy', optimizer=Adam())

# Load the discriminator model
discriminator_model = define_discriminator_model(image_shape)
discriminator_model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Get the list of image files in the input directory
image_files = os.listdir(input_directory)

# Create the output directory
os.makedirs(output_directory, exist_ok=True)

# Generate augmented images using the generator and discriminator models and save them to the output directory
for i, image_file in enumerate(image_files):
    image_path = os.path.join(input_directory, image_file)
    image = cv2.imread(image_path)  # Load the original image

    noise = np.random.normal(0, 1, (1, 100))  # Generate random noise vector of shape (1, 100)
    generated_image = generator_model.predict(noise)  # Use the generator model to generate an image

    # Combine the generated image with the original image
    combined_image = np.concatenate([image[np.newaxis, ...], generated_image], axis=0)

    # Label the images as real or fake (1 for real, 0 for fake)
    labels = np.array([1, 0])

    # Train the discriminator on the combined images and labels
    discriminator_model.train_on_batch(combined_image, labels)

    # Generate a new image using the generator model
    augmented_image = generator_model.predict(noise)  # Use the generator model to generate the final augmented image

    # Add minimal augmentation by blending the augmented image with the original image
    alpha = 0.2
    augmented_image = (augmented_image[0] * (1 - alpha) + image * alpha).astype(np.uint8)

    augmented_image_path = os.path.join(output_directory, f"augmented_image_{i}.png")
    cv2.imwrite(augmented_image_path, augmented_image)  # Save the augmented image

print("Augmentation complete.")


1/1 [==============================] - 0s 291ms/step
Augmentation complete.
